In [1]:
import pandas as pd
from collections import Counter
import numpy as np
from copy import deepcopy
import random
from functools import reduce
from itertools import chain, zip_longest
import pickle
import re
from sklearn.model_selection import train_test_split

# noisy testset augmentation

In [2]:
train_replace_dict = {'atis_flight#atis_airfare': 'atis_airfare', 
                                     'atis_ground_service#atis_ground_fare': 'atis_ground_fare',
                                     'atis_aircraft#atis_flight#atis_flight_no': 'atis_flight_no',
                                     'atis_airfare#atis_flight_time':'atis_flight_time',
                                     'atis_airline#atis_flight_no':'atis_flight_no',
                                     'atis_flight_no#atis_airline' : 'atis_flight_no',
                                     'atis_airfare#atis_flight': 'atis_airfare',
                                     'atis_flight#atis_airline' : 'atis_airline'}

# intent to idx mapping
intent_list = (pd.read_csv('../../data/ATIS/intent_list.tsv',sep=',',header=None,names=['INTENT']).INTENT.values.tolist())
intent2idx_ATIS = { intent:idx for idx,intent in enumerate(intent_list)}

# slot idx mapping
final_slots = pd.read_csv('../../data/ATIS/slots_list.tsv',sep=',',header=None,names=['SLOTS']).SLOTS.values.tolist()
slots2idx_ATIS  = {slots:idx for idx,slots in enumerate(final_slots)}
idx2slots = {v: k for k, v in slots2idx_ATIS.items()}

In [3]:
with open('../../data/BG_Noise_Phrase.txt') as f:
    content = f.readlines()

phrase = [x.strip() for x in content] 

In [4]:
def mapping2idx(df,intent2idx,slots2idx):

    # adding intent encoding
    intentID = []
    for intent in list(df.INTENT):
        try:
            intentID.append(intent2idx[intent])
        except:
            print(intent)
            intentID.append(intent2idx['UNK'])
            
        
    df['INTENT_ID'] = intentID 

    # adding mapped slot column
    slots_ID = []

    for annotations in list(df.SLOTS):
        slotID = ""

        for tokens in annotations.split():
            try: 
                slotID += str(slots2idx[tokens]) + " "
            except:
                print('token',tokens)
                slotID += str(slots2idx['O']) + " "
                
        slots_ID.append(" ".join(slotID.split()))

    df['SLOTS_ID'] = slots_ID
    
    return df

In [5]:
def merge_text_label(text,slot):
    data = []
    for i in range(len(text)):
        data.append([text[i] , slot[i]])
    return data

def interleave_lists(l1, l2 , prob):
    
    spl = [0]*int((1000)*prob) + [1]*int(1000*(1-prob))
    final = []
    while len(l1) >0 and len(l2) > 0:
        if random.sample(spl,1)[0] == 0:
            final.append(l1.pop(0))
        else:
            final.append(l2.pop(0))
    if len(l1) == 0:
        final = final + l2
    else:
        final = final + l1
    text,slot = zip(*final)
    return ' '.join(text), ' '.join(slot)

In [6]:
def BG_Noise(data, prob):
    
    orig_data = deepcopy(data)
    
    augINTENT, augSLOTS, augTEXT,augID = [],[],[],[]
    
    for idx,sample in enumerate(data.values.tolist()):
        
        TEXT, SLOTS = sample[1], sample[3]
        
        bg_TEXT = random.sample(phrase,1)[0]
        bg_SLOTS = 'O '*len(bg_TEXT.split())

        noisyData = merge_text_label(bg_TEXT.split(),bg_SLOTS.split())
        cleanData = merge_text_label(TEXT.split(),SLOTS.split())
        
        augText , augSlots = interleave_lists(noisyData,cleanData,prob)
        
        augINTENT.append(sample[2])
        augTEXT.append(augText)
        augSLOTS.append(augSlots)
        augID.append(idx)
               
    augPD = pd.DataFrame([augID,augTEXT,augINTENT,augSLOTS],index=['ID','TEXT','INTENT', 'SLOTS']).T
    
    return augPD
    

In [7]:
def ASRN_aug(data_path,out_path,fn,prob):
    
    # loading dataset
    data = pd.read_csv(data_path,sep='\t',header=0,names=['ID','TEXT','INTENT','SLOTS'])
    
    # lowerCasing the TEXT column
    data['TEXT'] = data['TEXT'].str.lower()
    
    # handling multi-label instances 
    data['INTENT'] = data['INTENT'].replace(train_replace_dict)
    
    aug_data = BG_Noise(data,prob)

    aug_data = mapping2idx(aug_data, intent2idx_ATIS ,slots2idx_ATIS)
        
    aug_data = aug_data.sample(frac=1).reset_index(drop=True)
    aug_data.to_csv(out_path+fn,sep='\t',index=False)
    

In [8]:
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/25per/','test_EN_01.tsv',0.25)
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/25per/','test_EN_02.tsv',0.25)
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/25per/','test_EN_03.tsv',0.25)
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/25per/','test_EN_04.tsv',0.25)
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/25per/','test_EN_05.tsv',0.25)
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/75per/','test_EN_01.tsv',0.75)
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/75per/','test_EN_02.tsv',0.75)
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/75per/','test_EN_03.tsv',0.75)
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/75per/','test_EN_04.tsv',0.75)
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/75per/','test_EN_05.tsv',0.75)
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/50per/','test_EN_01.tsv',0.50)
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/50per/','test_EN_02.tsv',0.50)
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/50per/','test_EN_03.tsv',0.50)
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/50per/','test_EN_04.tsv',0.50)
ASRN_aug('../../data/ATIS/raw/test/test.tsv','../../data/ATIS/experiments/test/NoiseBG/50per/','test_EN_05.tsv',0.50)

atis_day_name
atis_day_name
token I-flight_number
token B-compartment
token B-stoploc.airport_code
token I-state_name
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token I-flight_number
token B-compartment
token B-stoploc.airport_code
token I-state_name
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token I-flight_number
token B-compartment
token B-stoploc.airport_code
token I-state_name
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token I-flight_number
token B-compartment
token B-stoploc.airport_code
token I-state_name
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token I-flight_number
token B-compartment
token B-stoploc.airport_code
token I-state_name
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token I-flight_number
token B-compartment
token B-stoploc.airport_code
token I-state_name
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token I-flight_number
token B-compartment


# SNIPS noisy testset generation

In [9]:
# intent to idx mapping
intent_list = (pd.read_csv('../../data/SNIPS/intent_list.tsv',sep='\t',header=None,names=['INTENT']).INTENT.values.tolist())
intent2idx_SNIPS = { intent:idx for idx,intent in enumerate(intent_list)}

# slot idx mapping
final_slots = pd.read_csv('../../data/SNIPS/slots_list.tsv',sep='\t',header=None,names=['SLOTS']).SLOTS.values.tolist()
slots2idx_SNIPS  = {slots:idx for idx,slots in enumerate(final_slots)}
idx2slots = {v: k for k, v in slots2idx_SNIPS.items()}

FileNotFoundError: [Errno 2] No such file or directory: '../../data/SNIPS/slot_list.tsv'

In [10]:
def ASRN_aug(data_path,out_path,fn,prob):
    
    # loading dataset
    data = pd.read_csv(data_path,sep='\t',header=0,names=['ID','TEXT','INTENT','SLOTS'])
    
    # lowerCasing the TEXT column
    data['TEXT'] = data['TEXT'].str.lower()
    
    aug_data = BG_Noise(data,prob)

    aug_data = mapping2idx(aug_data, intent2idx_SNIPS ,slots2idx_SNIPS)
        
    aug_data = aug_data.sample(frac=1).reset_index(drop=True)
    aug_data.to_csv(out_path+fn,sep='\t',index=False)
    

In [23]:
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/25per/','test_EN_01.tsv',0.25)
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/25per/','test_EN_02.tsv',0.25)
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/25per/','test_EN_03.tsv',0.25)
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/25per/','test_EN_04.tsv',0.25)
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/25per/','test_EN_05.tsv',0.25)
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/75per/','test_EN_01.tsv',0.75)
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/75per/','test_EN_02.tsv',0.75)
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/75per/','test_EN_03.tsv',0.75)
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/75per/','test_EN_04.tsv',0.75)
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/75per/','test_EN_05.tsv',0.75)
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/50per/','test_EN_01.tsv',0.50)
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/50per/','test_EN_02.tsv',0.50)
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/50per/','test_EN_03.tsv',0.50)
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/50per/','test_EN_04.tsv',0.50)
ASRN_aug('../../data/SNIPS/raw/test/test.tsv','../../data/SNIPS/experiments/test/NoiseBG/50per/','test_EN_05.tsv',0.50)